In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor, VotingRegressor
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
from sklearn.linear_model import LinearRegression, Ridge, Lasso


In [4]:
sample_solution = pd.read_csv("133e814757ed11f0/dataset/sample_solution.csv", index_col='ID')

In [5]:
sample_solution.columns

Index(['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],
      dtype='object')

In [6]:
sample_solution

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,-0.117370,0.348090,0.473673,0.079501,-0.411504,0.015352,0.454957,0.065651,-0.146684,-0.140500
2,-0.503910,-0.250186,-1.412727,-0.523577,-0.577571,-0.294264,-1.396187,-0.856044,-0.003241,-0.246948
3,1.484172,1.272972,1.188539,1.321349,1.472491,1.237584,1.192748,1.575889,0.773926,1.917254
4,0.841616,0.457436,0.534375,0.376650,1.593406,0.157950,0.516430,0.632370,0.376289,-0.446052
5,-0.024147,0.136198,1.174866,-0.197264,2.463520,0.418315,1.185596,0.509797,-0.434762,0.807128
...,...,...,...,...,...,...,...,...,...,...
496,0.022706,-0.047897,0.474691,-0.016551,-0.431127,-0.081999,0.462777,0.089287,-0.608682,0.516307
497,-1.475840,-1.294104,-1.230311,-1.231267,-0.303306,-0.977209,-1.217209,-1.553847,-1.126938,-1.407277
498,1.002035,0.870014,0.037003,0.963333,-0.261270,0.620495,0.029488,0.414339,0.726121,-0.088445


In [7]:
train_dataset = pd.read_csv('133e814757ed11f0/dataset/train.csv')
test_dataset  = pd.read_csv('133e814757ed11f0/dataset/test.csv',index_col='ID')

In [8]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    2000 non-null   float64
 1   Component2_fraction    2000 non-null   float64
 2   Component3_fraction    2000 non-null   float64
 3   Component4_fraction    2000 non-null   float64
 4   Component5_fraction    2000 non-null   float64
 5   Component1_Property1   2000 non-null   float64
 6   Component2_Property1   2000 non-null   float64
 7   Component3_Property1   2000 non-null   float64
 8   Component4_Property1   2000 non-null   float64
 9   Component5_Property1   2000 non-null   float64
 10  Component1_Property2   2000 non-null   float64
 11  Component2_Property2   2000 non-null   float64
 12  Component3_Property2   2000 non-null   float64
 13  Component4_Property2   2000 non-null   float64
 14  Component5_Property2   2000 non-null   float64
 15  Comp

In [9]:
train_dataset, output_blends = train_dataset.iloc[:,:55],train_dataset.iloc[:,55:]

In [10]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    500 non-null    float64
 1   Component2_fraction    500 non-null    float64
 2   Component3_fraction    500 non-null    float64
 3   Component4_fraction    500 non-null    float64
 4   Component5_fraction    500 non-null    float64
 5   Component1_Property1   500 non-null    float64
 6   Component2_Property1   500 non-null    float64
 7   Component3_Property1   500 non-null    float64
 8   Component4_Property1   500 non-null    float64
 9   Component5_Property1   500 non-null    float64
 10  Component1_Property2   500 non-null    float64
 11  Component2_Property2   500 non-null    float64
 12  Component3_Property2   500 non-null    float64
 13  Component4_Property2   500 non-null    float64
 14  Component5_Property2   500 non-null    float64
 15  Component1_

In [11]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.480368,1.044967,-0.450956,0.674572,-0.636394,-1.244963,-1.355050,-0.314423,0.993593,-2.728928
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.958826,-0.019603,-0.807923,0.148715,1.439313,-1.160435,-0.014276,-0.135968,-1.221155,0.896222
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,-0.798978,-0.444027,0.148405,-0.793607,0.123834,0.006829,0.668734,0.015449,-0.098661,-0.424314
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.534135,1.155513,-0.760428,0.450159,-0.973779,0.052972,-1.024785,0.118951,2.400556,-0.576430
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.389350,1.799238,-0.912374,1.767557,-0.467038,2.104922,0.858593,-0.469110,0.715789,-2.038341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,1.138839,1.666804,-1.413339,0.405253,0.766653,-0.322096,1.399468,1.096369,-0.346225,0.641193
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.782418,0.784366,1.113626,1.328112,-2.537512,0.461525,0.647984,-0.618766,-0.047918,0.397253
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-0.813747,-0.197880,-0.549162,0.810814,1.567580,-0.694918,-1.710215,-0.233936,-0.133002,-0.284672
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,1.262477,-0.925444,-0.823345,0.427648,-0.161447,0.628131,-0.038484,0.343058,0.448748,0.193507


In [12]:
output_blends

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073
...,...,...,...,...,...,...,...,...,...,...
1995,-0.028366,-0.327297,-0.316933,-1.294092,-0.530259,-0.421526,-0.320869,0.709627,-0.737244,-0.744289
1996,-0.449245,0.156778,-0.367445,-0.938615,-0.577451,-0.209996,-0.370505,-0.195531,-0.032834,0.269718
1997,0.029135,0.164890,-0.092942,-1.134490,-0.437479,-0.695636,-0.101073,0.063650,0.624368,-0.477053
1998,-0.232960,-0.464947,0.112536,-0.793522,-0.811272,-1.194914,0.100644,0.760116,-0.751394,-0.857598


In [13]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ------------------------------------------------------------------
# 0.  LOAD & COPY ORIGINAL DATAFRAME
# ------------------------------------------------------------------
# assume your raw dataframe is named `df`
# train_dataset = df.copy()

# ------------------------------------------------------------------
# 1.  COLUMN SET‑UP
# ------------------------------------------------------------------
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]

property_cols = {}
for prop_idx in range(1, 11):
    property_cols[prop_idx] = [
        f'Component{i}_Property{prop_idx}' for i in range(1, 6)
    ]

# ------------------------------------------------------------------
# 2.  LINEAR BLEND FEATURES
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    train_dataset[w_col] = 0.0
    for comp_idx in range(1, 6):
        train_dataset[w_col] += (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )
    for comp_idx in range(1, 6):
        train_dataset[f'Deviation_{comp_idx}_Prop{prop_idx}'] = (
            train_dataset[f'Component{comp_idx}_Property{prop_idx}']
            - train_dataset[w_col]
        )

# ------------------------------------------------------------------
# 3.  FRACTION‑BASED FEATURES
# ------------------------------------------------------------------
train_dataset['Fraction_Entropy'] = (
    -train_dataset[fraction_cols] * np.log(train_dataset[fraction_cols] + 1e-10)
).sum(axis=1)
train_dataset['Dominant_Fraction'] = train_dataset[fraction_cols].max(axis=1)
train_dataset['Fraction_Range']    = (
    train_dataset[fraction_cols].max(axis=1)
    - train_dataset[fraction_cols].min(axis=1)
)

for i in range(1, 6):
    for j in range(i + 1, 6):
        train_dataset[f'Frac_Interaction_{i}_{j}'] = (
            train_dataset[f'Component{i}_fraction']
            * train_dataset[f'Component{j}_fraction']
        )

# ------------------------------------------------------------------
# 4.  PROPERTY INTERACTION FEATURES
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    train_dataset[f'Prop{prop_idx}_Min']      = train_dataset[cols].min(axis=1)
    train_dataset[f'Prop{prop_idx}_Max']      = train_dataset[cols].max(axis=1)
    train_dataset[f'Prop{prop_idx}_Range']    = (
        train_dataset[f'Prop{prop_idx}_Max'] - train_dataset[f'Prop{prop_idx}_Min']
    )
    train_dataset[f'Prop{prop_idx}_Variance'] = train_dataset[cols].var(axis=1)

    max_vals = train_dataset[cols].max(axis=1)
    for comp_idx in range(1, 6):
        train_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
            train_dataset[cols[comp_idx - 1]] == max_vals
        ).astype(int)

# ------------------------------------------------------------------
# 5.  CROSS‑PROPERTY INTERACTIONS
# ------------------------------------------------------------------
for comp_idx in range(1, 6):
    comp_props = [f'Component{comp_idx}_Property{p}' for p in range(1, 11)]
    train_dataset[f'Comp{comp_idx}_Prop_Mean'] = train_dataset[comp_props].mean(axis=1)
    train_dataset[f'Comp{comp_idx}_Prop_Std']  = train_dataset[comp_props].std(axis=1)

    for prop_idx in range(1, 11):
        train_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Scaled'] = (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )

# ------------------------------------------------------------------
# 6.  NON‑LINEAR TRANSFORMATIONS
# ------------------------------------------------------------------
for comp_idx in range(1, 6):
    frac_col = f'Component{comp_idx}_fraction'
    train_dataset[f'{frac_col}_Sq']   = train_dataset[frac_col] ** 2
    train_dataset[f'{frac_col}_Sqrt'] = np.sqrt(train_dataset[frac_col])

    for prop_idx in range(1, 11):
        col = f'Component{comp_idx}_Property{prop_idx}'
        train_dataset[f'{col}_Log'] = np.log1p(train_dataset[col])
        train_dataset[f'{col}_Sq']  = train_dataset[col] ** 2

# ------------------------------------------------------------------
# 7.  MIXTURE CHARACTERISTICS
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    max_val = train_dataset[cols].max(axis=1)
    min_val = train_dataset[cols].min(axis=1)
    train_dataset[f'Prop{prop_idx}_BlendRatio'] = (
        (max_val - min_val) / (max_val + 1e-10)
    )

    for comp_idx in range(1, 6):
        col = f'Component{comp_idx}_Property{prop_idx}'
        train_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Adj'] = (
            train_dataset[col] * (1 + train_dataset[f'Component{comp_idx}_fraction'])
        )

# ------------------------------------------------------------------
# 8.  CLUSTER LABEL (optional, no target leak)
# ------------------------------------------------------------------
cluster_features = [f'Weighted_Prop{i}' for i in range(1, 11)] + fraction_cols
kmeans = KMeans(n_clusters=5, random_state=42)
# fit on train
_ = kmeans.fit(train_dataset[cluster_features])

# train: compute distances to each center
train_dists = kmeans.transform(train_dataset[cluster_features])
train_dist_df = pd.DataFrame(
    train_dists,
    columns=[f'Cluster_Dist_{i}' for i in range(5)],
    index=train_dataset.index
)
train_dataset = pd.concat([train_dataset, train_dist_df], axis=1)
# ------------------------------------------------------------------
# 9.  TARGET‑INDEPENDENT CONTRIBUTIONS
# ------------------------------------------------------------------
for target_idx in range(1, 11):
    w_col = f'Weighted_Prop{target_idx}'
    for comp_idx in range(1, 6):
        contrib_col = f'Comp{comp_idx}_Contrib_Prop{target_idx}'
        train_dataset[contrib_col] = (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{target_idx}']
            / (train_dataset[w_col] + 1e-10)
        )

# ------------------------------------------------------------------
# 10.  CLEAN‑UP
# ------------------------------------------------------------------
train_dataset.drop(columns=['Blend_Cluster'], inplace=True)


/tmp/ipykernel_210304/4276239847.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
/tmp/ipykernel_210304/4276239847.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_dataset[f'Prop{prop_idx}_Min']      = train_dataset[cols].min(axis=1)
/tmp/ipykernel_210304/4276239847.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

KeyError: "['Blend_Cluster'] not found in axis"

In [ ]:
train_dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ------------------------------------------------------------
# 0. COPY RAW TEST DATA
# ------------------------------------------------------------
# assume your test dataframe is named `test_df`
# test_dataset = test_df.copy()

# ------------------------------------------------------------
# 1. COLUMN SETUP
# ------------------------------------------------------------
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]
property_cols = {
    prop_idx: [f'Component{i}_Property{prop_idx}' for i in range(1, 6)]
    for prop_idx in range(1, 11)
}

# ------------------------------------------------------------
# 2. LINEAR BLEND FEATURES
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    test_dataset[w_col] = 0.0
    for comp_idx in range(1, 6):
        test_dataset[w_col] += (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )
    for comp_idx in range(1, 6):
        test_dataset[f'Deviation_{comp_idx}_Prop{prop_idx}'] = (
            test_dataset[f'Component{comp_idx}_Property{prop_idx}']
            - test_dataset[w_col]
        )

# ------------------------------------------------------------
# 3. FRACTION-BASED FEATURES
# ------------------------------------------------------------
test_dataset['Fraction_Entropy'] = (
    -test_dataset[fraction_cols] * np.log(test_dataset[fraction_cols] + 1e-10)
).sum(axis=1)
test_dataset['Dominant_Fraction'] = test_dataset[fraction_cols].max(axis=1)
test_dataset['Fraction_Range']    = (
    test_dataset[fraction_cols].max(axis=1)
    - test_dataset[fraction_cols].min(axis=1)
)

for i in range(1, 6):
    for j in range(i + 1, 6):
        test_dataset[f'Frac_Interaction_{i}_{j}'] = (
            test_dataset[f'Component{i}_fraction']
            * test_dataset[f'Component{j}_fraction']
        )

# ------------------------------------------------------------
# 4. PROPERTY INTERACTION FEATURES
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    test_dataset[f'Prop{prop_idx}_Min']      = test_dataset[cols].min(axis=1)
    test_dataset[f'Prop{prop_idx}_Max']      = test_dataset[cols].max(axis=1)
    test_dataset[f'Prop{prop_idx}_Range']    = (
        test_dataset[f'Prop{prop_idx}_Max'] - test_dataset[f'Prop{prop_idx}_Min']
    )
    test_dataset[f'Prop{prop_idx}_Variance'] = test_dataset[cols].var(axis=1)

    max_vals = test_dataset[cols].max(axis=1)
    for comp_idx in range(1, 6):
        test_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
            test_dataset[cols[comp_idx - 1]] == max_vals
        ).astype(int)

# ------------------------------------------------------------
# 5. CROSS-PROPERTY INTERACTIONS
# ------------------------------------------------------------
for comp_idx in range(1, 6):
    comp_props = [f'Component{comp_idx}_Property{p}' for p in range(1, 11)]
    test_dataset[f'Comp{comp_idx}_Prop_Mean'] = test_dataset[comp_props].mean(axis=1)
    test_dataset[f'Comp{comp_idx}_Prop_Std']  = test_dataset[comp_props].std(axis=1)

    for prop_idx in range(1, 11):
        test_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Scaled'] = (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )

# ------------------------------------------------------------
# 6. NON-LINEAR TRANSFORMATIONS
# ------------------------------------------------------------
for comp_idx in range(1, 6):
    frac_col = f'Component{comp_idx}_fraction'
    test_dataset[f'{frac_col}_Sq']   = test_dataset[frac_col] ** 2
    test_dataset[f'{frac_col}_Sqrt'] = np.sqrt(test_dataset[frac_col])

    for prop_idx in range(1, 11):
        col = f'Component{comp_idx}_Property{prop_idx}'
        test_dataset[f'{col}_Log'] = np.log1p(test_dataset[col])
        test_dataset[f'{col}_Sq']  = test_dataset[col] ** 2

# ------------------------------------------------------------
# 7. MIXTURE CHARACTERISTICS
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    max_val = test_dataset[cols].max(axis=1)
    min_val = test_dataset[cols].min(axis=1)
    test_dataset[f'Prop{prop_idx}_BlendRatio'] = (
        (max_val - min_val) / (max_val + 1e-10)
    )

    for comp_idx in range(1, 6):
        col = f'Component{comp_idx}_Property{prop_idx}'
        test_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Adj'] = (
            test_dataset[col] * (1 + test_dataset[f'Component{comp_idx}_fraction'])
        )

# ------------------------------------------------------------
# 8. OPTIONAL: CLUSTER LABEL (no target leak)
# ------------------------------------------------------------
cluster_features = [f'Weighted_Prop{i}' for i in range(1, 11)] + fraction_cols

# test: compute distances
test_dists = kmeans.transform(test_dataset[cluster_features])
test_dist_df = pd.DataFrame(
    test_dists,
    columns=[f'Cluster_Dist_{i}' for i in range(5)],
    index=test_dataset.index
)
test_dataset = pd.concat([test_dataset, test_dist_df], axis=1)

# ------------------------------------------------------------
# 9. TARGET-INDEPENDENT CONTRIBUTIONS
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    for comp_idx in range(1, 6):
        contrib_col = f'Comp{comp_idx}_Contrib_Prop{prop_idx}'
        test_dataset[contrib_col] = (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
            / (test_dataset[w_col] + 1e-10)
        )

# ------------------------------------------------------------
# 10. CLEAN-UP
# ------------------------------------------------------------
test_dataset.drop(columns=['Blend_Cluster'], inplace=True)


#  Add More Featurs

In [ ]:
test_dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

def create_blending_features(df):
    # Create a copy to avoid modifying original data
    df = df.copy()
    
    # Assumptions about property indices (based on petroleum blending domain knowledge):
    # Property1: API Gravity (density measure)
    # Property2: RVP (Reid Vapor Pressure)
    # Property3: RON (Research Octane Number)
    # Property4: Olefins content
    # Property5: Aromatics content
    # Property6: T10 (10% distillation temperature)
    # Property7: T50 (50% distillation temperature)
    # Property8: T90 (90% distillation temperature)
    # Property9: Viscosity
    # Property10: Cetane Index
    
    # 1. Density and Gravity Features
    for i in range(1, 6):
        # Convert API to Specific Gravity (60/60)
        df[f'Component{i}_SG'] = 141.5 / (df[f'Component{i}_Property1'] + 131.5)
        
        # Calculate density blending index
        df[f'Component{i}_Density_Index'] = df[f'Component{i}_SG'] ** (-0.06)
    
    # Blend density index volumetrically
    df['Blend_Density_Index'] = 0
    for i in range(1, 6):
        df['Blend_Density_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Density_Index']
    
    # Convert back to specific gravity
    df['Blend_SG'] = df['Blend_Density_Index'] ** (-1/0.06)
    df['Blend_API'] = (141.5 / df['Blend_SG']) - 131.5
    
    # 2. Sulfur Blending (gravimetric) - assuming Property2 is sulfur-related
    df['Blend_Sulfur'] = 0
    for i in range(1, 6):
        df['Blend_Sulfur'] += (df[f'Component{i}_fraction'] * df[f'Component{i}_SG'] * 
                              df[f'Component{i}_Property2']) / df['Blend_SG']
    
    # 3. RVP Blending (Index Method)
    rvp_exponent = -0.07  # Texaco method exponent
    for i in range(1, 6):
        df[f'Component{i}_RVP_Index'] = df[f'Component{i}_Property2'] ** rvp_exponent
    
    df['Blend_RVP_Index'] = 0
    for i in range(1, 6):
        df['Blend_RVP_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_RVP_Index']
    
    df['Blend_RVP'] = df['Blend_RVP_Index'] ** (1/rvp_exponent)
    
    # 4. Octane Blending (Ethyl RT-70 method)
    for i in range(1, 6):
        # Simplified octane interaction term
        df[f'Component{i}_RON_BlendValue'] = (
            df[f'Component{i}_Property3'] + 
            0.1 * df[f'Component{i}_Property4'] - 
            0.05 * df[f'Component{i}_Property5']
        )
    
    df['Blend_RON'] = 0
    for i in range(1, 6):
        df['Blend_RON'] += df[f'Component{i}_fraction'] * df[f'Component{i}_RON_BlendValue']
    
    # 5. Distillation Blending (Ethyl S-Curve Model)
    distillation_points = {'T10': 6, 'T50': 7, 'T90': 8}
    for point, prop_idx in distillation_points.items():
        for i in range(1, 6):
            # Simplified distillation blending index
            df[f'Component{i}_{point}_Index'] = np.log(df[f'Component{i}_Property{prop_idx}'])
        
        df[f'Blend_{point}_Index'] = 0
        for i in range(1, 6):
            df[f'Blend_{point}_Index'] += (
                df[f'Component{i}_fraction'] * 
                df[f'Component{i}_{point}_Index']
            )
        
        df[f'Blend_{point}'] = np.exp(df[f'Blend_{point}_Index'])
    
    # 6. Cetane Index (ASTM D-976)
    for i in range(1, 6):
        df[f'Component{i}_Cetane_Index'] = (
            420.34 + 0.016 * df[f'Component{i}_Property1']**2 + 
            0.192 * df[f'Component{i}_Property1'] * np.log(df[f'Component{i}_Property7']) + 
            65.01 * (np.log(df[f'Component{i}_Property7']))**2 - 
            0.0001809 * df[f'Component{i}_Property7']**2
        )
    
    df['Blend_Cetane'] = 0
    for i in range(1, 6):
        df['Blend_Cetane'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Cetane_Index']
    
    # 7. Viscosity Blending
    for i in range(1, 6):
        df[f'Component{i}_Visc_Index'] = np.log(np.log(df[f'Component{i}_Property9'] + 0.8))
    
    df['Blend_Visc_Index'] = 0
    for i in range(1, 6):
        df['Blend_Visc_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Visc_Index']
    
    df['Blend_Viscosity'] = np.exp(np.exp(df['Blend_Visc_Index'])) - 0.8
    
    # # 8. Interaction Terms
    # for i in range(1, 6):
    #     for j in range(i+1, 6):
    #         df[f'Frac_Interaction_{i}_{j}'] = (
    #             df[f'Component{i}_fraction'] * 
    #             df[f'Component{j}_fraction']
    #         )
    
    # 9. Polynomial Features
    poly = PolynomialFeatures(degree=2, include_bias=False)
    component_features = [f'Component{i}_fraction' for i in range(1, 6)] + \
                         [f'Component{i}_Property1' for i in range(1, 6)] + \
                         [f'Component{i}_Property2' for i in range(1, 6)]
    
    poly_features = poly.fit_transform(df[component_features])
    poly_cols = [f'Poly_{i}' for i in range(poly_features.shape[1])]
    poly_df = pd.DataFrame(poly_features, columns=poly_cols)
    df = pd.concat([df, poly_df], axis=1)
    
    # # 10. Mixture Complexity Metrics
    # fractions = [f'Component{i}_fraction' for i in range(1, 6)]
    # df['Fraction_Entropy'] = (-df[fractions] * np.log(df[fractions] + 1e-9)).sum(axis=1)
    # df['Dominant_Fraction'] = df[fractions].max(axis=1)
    
    # 11. Final Cleaning
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)
    
    return df

# Apply feature engineering to both datasets
train_dataset = create_blending_features(train_dataset)
test_dataset = create_blending_features(test_dataset)

In [ ]:
train_dataset

In [ ]:
test_dataset

In [14]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Comp1_Contrib_Prop9,Comp2_Contrib_Prop9,Comp3_Contrib_Prop9,Comp4_Contrib_Prop9,Comp5_Contrib_Prop9,Comp1_Contrib_Prop10,Comp2_Contrib_Prop10,Comp3_Contrib_Prop10,Comp4_Contrib_Prop10,Comp5_Contrib_Prop10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,26.889721,0.000000,-50.486585,44.953230,-20.356366,0.553231,0.000000,0.279444,-0.525630,0.692954
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,0.299647,0.049479,1.174108,-0.523234,-0.000000,0.037708,0.007654,0.041973,0.912665,-0.000000
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,0.142130,0.078988,-0.059399,0.882339,-0.044057,-0.009046,-0.885818,-0.046045,0.816799,1.124109
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-1.026313,3.730034,-0.000000,0.242188,-1.945908,-0.033191,1.078722,-0.000000,-0.421151,0.375619
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.101491,0.288618,-0.073178,0.886051,-0.000000,0.544546,0.136689,-0.037341,0.356107,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,0.588973,0.206885,-0.000000,0.108984,0.095158,26.030345,-27.143699,-0.000000,14.549770,-12.436416
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.580659,0.949748,0.000000,1.919396,-1.288486,0.271925,0.622911,-0.000000,-0.054979,0.160144
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-11.982363,-0.460069,-2.979187,9.739846,6.681772,0.559495,0.217411,0.069391,0.087357,0.066346
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,1.181403,-0.277124,-0.000000,0.144067,-0.048345,0.748410,-0.014673,0.000000,0.192484,0.073780


In [15]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,-0.265376,0.123432,0.028533,-0.173365,1.297923,0.323299,-0.315146,0.625518,-0.514342,-0.777057
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,-0.787677,-0.757905,-0.280561,-1.965970,0.543475,-0.906851,0.962341,-0.183757,0.310871,-1.329042
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,-0.710026,-1.422693,0.874071,-1.016144,0.093525,1.048525,-1.321851,0.356640,-0.869543,-0.177255
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,-0.551366,0.257105,-0.077337,-0.721031,-0.760365,-0.507690,1.346556,-0.001529,-1.008445,1.726105
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,-1.811468,-0.181223,-0.475933,0.234775,-0.909020,1.238203,-1.805664,0.980417,-1.354932,-0.657513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,-2.690249,-0.231333,-1.365939,-0.853755,-1.344675,-1.947526,-1.074584,-0.421069,-0.603527,0.838356
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,-0.458610,-1.422693,1.952542,-0.283461,0.367323,2.689269,1.698609,-0.328886,-0.879281,-1.658223
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,-0.437798,-0.810355,-0.926471,-1.480680,-1.613630,1.016878,0.989316,0.408454,-0.925924,-0.022020


In [16]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    500 non-null    float64
 1   Component2_fraction    500 non-null    float64
 2   Component3_fraction    500 non-null    float64
 3   Component4_fraction    500 non-null    float64
 4   Component5_fraction    500 non-null    float64
 5   Component1_Property1   500 non-null    float64
 6   Component2_Property1   500 non-null    float64
 7   Component3_Property1   500 non-null    float64
 8   Component4_Property1   500 non-null    float64
 9   Component5_Property1   500 non-null    float64
 10  Component1_Property2   500 non-null    float64
 11  Component2_Property2   500 non-null    float64
 12  Component3_Property2   500 non-null    float64
 13  Component4_Property2   500 non-null    float64
 14  Component5_Property2   500 non-null    float64
 15  Component1_

# polynomial _Features creation

In [17]:
train_dataset.corr()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Comp1_Contrib_Prop9,Comp2_Contrib_Prop9,Comp3_Contrib_Prop9,Comp4_Contrib_Prop9,Comp5_Contrib_Prop9,Comp1_Contrib_Prop10,Comp2_Contrib_Prop10,Comp3_Contrib_Prop10,Comp4_Contrib_Prop10,Comp5_Contrib_Prop10
Component1_fraction,1.000000,-0.452466,-0.441056,-0.128280,0.028830,-0.015530,-0.032505,-0.000743,0.006344,0.042178,...,-0.026935,0.026826,-0.028990,0.004341,0.032009,-0.000744,-0.009004,0.001944,-0.001894,-0.003541
Component2_fraction,-0.452466,1.000000,-0.430003,-0.162097,0.056293,0.024858,0.020551,-0.039978,-0.007805,-0.025751,...,0.003454,0.006005,0.004053,-0.008347,-0.007474,-0.012064,-0.012269,0.011958,-0.011880,-0.011041
Component3_fraction,-0.441056,-0.430003,1.000000,-0.205391,0.063267,-0.024512,0.010782,0.030103,-0.001353,-0.005236,...,0.008856,-0.017125,0.016617,0.007719,-0.014218,0.028516,0.034102,-0.028772,0.028018,0.029061
Component4_fraction,-0.128280,-0.162097,-0.205391,1.000000,-0.741647,0.017562,0.003803,0.014451,0.005666,-0.008616,...,0.025247,-0.030996,0.004608,0.014787,-0.026588,-0.034140,-0.027942,0.032977,-0.032234,-0.032290
Component5_fraction,0.028830,0.056293,0.063267,-0.741647,1.000000,0.000784,-0.004848,-0.004726,-0.004142,-0.007248,...,-0.015021,0.023196,0.008156,-0.033809,0.026377,0.027082,0.021820,-0.026729,0.026725,0.026301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Comp1_Contrib_Prop10,-0.000744,-0.012064,0.028516,-0.034140,0.027082,0.033461,0.021252,-0.040317,-0.004093,0.012714,...,0.000614,-0.000670,0.001340,-0.000836,-0.000429,1.000000,0.908402,-0.999773,0.998370,0.997482
Comp2_Contrib_Prop10,-0.009004,-0.012269,0.034102,-0.027942,0.021820,0.033302,0.016711,-0.025561,-0.008087,-0.003513,...,-0.000083,-0.008008,-0.010395,0.004649,0.011054,0.908402,1.000000,-0.909706,0.891217,0.905080
Comp3_Contrib_Prop10,0.001944,0.011958,-0.028772,0.032977,-0.026729,-0.034173,-0.020783,0.040196,0.003169,-0.011962,...,-0.000653,0.000512,-0.001528,0.001147,0.000504,-0.999773,-0.909706,1.000000,-0.998862,-0.997822
Comp4_Contrib_Prop10,-0.001894,-0.011880,0.028018,-0.032234,0.026725,0.034545,0.020247,-0.041133,-0.001893,0.013034,...,0.000828,0.000448,0.003280,-0.002253,-0.001938,0.998370,0.891217,-0.998862,1.000000,0.996167


In [18]:
test_dataset.corr()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
Component1_fraction,1.000000,-0.417048,-0.471631,-0.167389,-0.008025,-0.027875,0.053371,-0.033673,0.021528,-0.036265,...,0.035460,-0.004974,0.067548,0.023547,-0.036902,-0.063037,0.010187,0.005433,0.016961,0.021979
Component2_fraction,-0.417048,1.000000,-0.414109,-0.232240,0.127492,-0.066247,-0.036146,-0.030215,-0.055298,0.030308,...,-0.034337,-0.010603,-0.038681,0.017296,0.000011,0.049003,0.056619,0.043948,0.008355,0.011256
Component3_fraction,-0.471631,-0.414109,1.000000,-0.127871,0.002758,0.022666,-0.008772,0.052068,-0.016428,0.013606,...,-0.022796,-0.009756,-0.011946,-0.033444,0.041452,0.038046,-0.043554,-0.080307,0.012460,0.008129
Component4_fraction,-0.167389,-0.232240,-0.127871,1.000000,-0.693851,0.082520,-0.042324,0.017643,0.066503,-0.013267,...,-0.004290,0.023893,-0.065110,-0.005124,-0.000027,-0.021319,-0.014685,0.017600,-0.031294,-0.080232
Component5_fraction,-0.008025,0.127492,0.002758,-0.693851,1.000000,0.000474,0.053983,-0.006125,-0.017857,0.011192,...,0.049847,0.010044,0.076337,-0.007056,-0.007328,-0.006356,-0.020817,0.034628,-0.023654,0.056889
Component1_Property1,-0.027875,-0.066247,0.022666,0.082520,0.000474,1.000000,0.015506,-0.052138,-0.059622,-0.078704,...,-0.041366,0.005612,0.027429,-0.000790,0.082723,0.010878,-0.037019,-0.001421,0.013622,0.011839
Component2_Property1,0.053371,-0.036146,-0.008772,-0.042324,0.053983,0.015506,1.000000,-0.008209,-0.010579,-0.067575,...,0.005894,0.018944,0.071292,-0.025284,0.045990,0.026087,-0.038798,-0.050381,-0.013059,0.054005
Component3_Property1,-0.033673,-0.030215,0.052068,0.017643,-0.006125,-0.052138,-0.008209,1.000000,0.050531,0.006698,...,-0.060875,0.004761,-0.026302,-0.025402,0.053931,-0.038322,-0.069412,0.023447,-0.016970,0.014580
Component4_Property1,0.021528,-0.055298,-0.016428,0.066503,-0.017857,-0.059622,-0.010579,0.050531,1.000000,-0.037595,...,0.006105,0.062234,-0.095138,-0.009429,-0.068668,-0.037017,0.048248,0.002843,0.003260,0.001527
Component5_Property1,-0.036265,0.030308,0.013606,-0.013267,0.011192,-0.078704,-0.067575,0.006698,-0.037595,1.000000,...,-0.019079,0.017183,0.013760,-0.011517,-0.007093,0.007039,-0.090300,-0.009376,-0.046132,-0.075517


In [19]:
print(list(train_dataset.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 317, 0, 323, 0, 300, 0, 317, 0, 315, 0, 306, 0, 321, 0, 323, 0, 332, 0, 300, 0, 0, 0, 374, 0, 306, 0, 348, 0, 319, 0, 327, 0, 335, 0, 332, 0, 331, 0, 400, 0, 3

In [20]:
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# train_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# poly = PolynomialFeatures(degree=(2),interaction_only=True,include_bias=False)
# poly.fit(train_dataset)

In [21]:
# from sklearn.preprocessing import StandardScaler

In [22]:
# train_dataset = poly.transform(train_dataset)
# scaler = StandardScaler().fit(train_dataset)
# train_dataset = scaler.transform(train_dataset)

In [23]:
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset = poly.transform(test_dataset)
# test_dataset = scaler.transform(test_dataset)

In [24]:
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')
# train_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')
# poly = PolynomialFeatures(degree=(2),include_bias=False)
# poly.fit(train_dataset.iloc[:,:55])
# train_log = np.log1p(np.maximum(train_dataset.iloc[:, :55], 1e-6))  

# train_dataset_55 = poly.transform(train_dataset.iloc[:,:55])
# train_dataset = np.hstack([train_dataset_55, train_log,  np.array(train_dataset.iloc[:,55:])])
# scaler = StandardScaler().fit(train_dataset)
# train_dataset = pd.DataFrame(scaler.transform(train_dataset))

In [25]:
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_log = np.log1p(np.maximum(test_dataset.iloc[:, :55], 1e-6))

# test_dataset_55 = poly.transform(test_dataset.iloc[:,:55])
# test_dataset = pd.DataFrame(np.hstack([test_dataset_55, test_log, np.array(test_dataset.iloc[:,55:])]))
# test_dataset = pd.DataFrame(scaler.transform(test_dataset))

In [26]:
# from sklearn.preprocessing import StandardScaler

# # Fix infinities and NaNs
# train_dataset = train_dataset.replace([np.inf, -np.inf], np.nan)
# train_dataset = train_dataset.interpolate(method='linear', limit_direction='both')
# train_dataset = train_dataset.fillna(train_dataset.mean())

# test_dataset = test_dataset.replace([np.inf, -np.inf], np.nan)
# test_dataset = test_dataset.interpolate(method='linear', limit_direction='both')
# test_dataset = test_dataset.fillna(test_dataset.mean())

# # Apply log transform only to relevant features
# log_cols = [col for col in train_dataset.columns if 'fraction' in col or 'Property' in col]
# for col in log_cols:
#     train_dataset[col] = np.log1p(np.maximum(train_dataset[col], 1e-6))
#     test_dataset[col]  = np.log1p(np.maximum(test_dataset[col], 1e-6))

# # Normalize everything
# scaler = StandardScaler()
# train_dataset = pd.DataFrame(scaler.fit_transform(train_dataset), columns=train_dataset.columns)
# test_dataset  = pd.DataFrame(scaler.transform(test_dataset), columns=test_dataset.columns)


In [27]:
from sklearn.feature_selection import VarianceThreshold

# --- 1. Drop near-zero variance features ---
# Threshold is the variance; tune as needed (e.g. 1e-5)
var_selector = VarianceThreshold(threshold=1e-5)
var_selector.fit(train_dataset)

# Keep the mask of good features
mask = var_selector.get_support()
selected_features = train_dataset.columns[mask]

# Reduce both datasets to those features
train_dataset = train_dataset[selected_features]
test_dataset  = test_dataset[selected_features]

# --- 2. Drop highly correlated features ---
# Compute correlation matrix on train only
corr_matrix = train_dataset.corr().abs()
upper_tri  = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Identify columns to drop (corr > 0.98)
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.98)]

# Drop them
train_dataset = train_dataset.drop(columns=to_drop)
test_dataset  = test_dataset.drop(columns=to_drop)

print(f"Dropped {len(to_drop)} correlated features: {to_drop}")
print(f"Final feature count: {train_dataset.shape[1]}")


KeyError: "['Weighted_Prop1', 'Deviation_1_Prop1', 'Deviation_2_Prop1', 'Deviation_3_Prop1', 'Deviation_4_Prop1', 'Deviation_5_Prop1', 'Weighted_Prop2', 'Deviation_1_Prop2', 'Deviation_2_Prop2', 'Deviation_3_Prop2', 'Deviation_4_Prop2', 'Deviation_5_Prop2', 'Weighted_Prop3', 'Deviation_1_Prop3', 'Deviation_2_Prop3', 'Deviation_3_Prop3', 'Deviation_4_Prop3', 'Deviation_5_Prop3', 'Weighted_Prop4', 'Deviation_1_Prop4', 'Deviation_2_Prop4', 'Deviation_3_Prop4', 'Deviation_4_Prop4', 'Deviation_5_Prop4', 'Weighted_Prop5', 'Deviation_1_Prop5', 'Deviation_2_Prop5', 'Deviation_3_Prop5', 'Deviation_4_Prop5', 'Deviation_5_Prop5', 'Weighted_Prop6', 'Deviation_1_Prop6', 'Deviation_2_Prop6', 'Deviation_3_Prop6', 'Deviation_4_Prop6', 'Deviation_5_Prop6', 'Weighted_Prop7', 'Deviation_1_Prop7', 'Deviation_2_Prop7', 'Deviation_3_Prop7', 'Deviation_4_Prop7', 'Deviation_5_Prop7', 'Weighted_Prop8', 'Deviation_1_Prop8', 'Deviation_2_Prop8', 'Deviation_3_Prop8', 'Deviation_4_Prop8', 'Deviation_5_Prop8', 'Wei

In [203]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Poly_121,Poly_122,Poly_123,Poly_124,Poly_126,Poly_127,Poly_128,Poly_130,Poly_131,Poly_133
1,0.18000,0.0500,0.3200,0.37000,0.0800,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,-1.810081,3.498733,2.146469,-0.506063,-2.559194,-1.570063,0.370166,3.034798,-0.715500,-0.438958
2,0.00000,0.5000,0.0000,0.37000,0.1300,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,-0.694796,-0.541017,0.211363,0.325443,2.768702,-1.081669,-1.665482,-0.842264,-1.296862,0.506655
3,0.16000,0.0000,0.1700,0.50000,0.1700,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,-0.435899,0.310553,-0.184379,-0.081505,-3.043598,1.807018,0.798800,-1.287395,-0.569098,0.337880
4,0.50000,0.0000,0.1700,0.16000,0.1700,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,-0.135313,-0.530006,-2.986600,1.174832,0.050454,0.284311,-0.111839,1.113617,-0.438061,-2.468484
5,0.00000,0.0000,0.5000,0.50000,0.0000,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,0.115426,-0.322387,-0.155090,-0.000328,-1.314205,-0.632221,-0.001336,1.765795,0.003731,0.001795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,0.19000,0.4700,0.0300,0.23000,0.0800,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,0.450171,-0.362829,1.865646,2.441252,-0.044423,0.228421,0.298896,-0.184103,-0.240904,1.238716
498,0.43000,0.0100,0.1200,0.21000,0.2300,0.806590,0.607324,0.359058,0.283394,1.032029,...,-1.204960,1.330293,-2.123274,1.170996,-1.894805,3.024289,-1.667910,-3.338856,1.841395,-2.939042
499,0.03000,0.0400,0.4200,0.42000,0.0900,-0.792140,0.674275,-1.783487,0.848296,0.164798,...,1.429071,1.129453,-0.264935,-1.117804,2.093854,-0.491153,-2.072257,-0.388179,-1.637789,0.384175
500,0.00000,0.5000,0.0000,0.50000,0.0000,-0.327778,0.248042,-1.199065,1.845241,0.772672,...,-0.011366,0.052772,-0.005110,-0.017928,0.016023,0.053878,-0.038085,-0.079114,0.018001,-0.040685


In [204]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 568 entries, Component1_fraction to Poly_133
dtypes: float64(518), int64(50)
memory usage: 8.7 MB


## Splitting the  test data and make testset and validationset

In [205]:
train_X, val_X, train_y, val_y = train_test_split(train_dataset,output_blends, random_state=42, test_size=0.01,)

In [206]:
# sns.pairplot(train_dataset)
print("Train Shape:", train_dataset.shape)
print("Test Shape :", test_dataset.shape)
print("Any NaNs:", train_dataset.isna().sum().sum(), test_dataset.isna().sum().sum())
print("Any Infs:", np.isinf(train_dataset.to_numpy()).sum(), np.isinf(test_dataset.to_numpy()).sum())


Train Shape: (2000, 568)
Test Shape : (501, 568)
Any NaNs: 0 0
Any Infs: 0 0


# Use Tablenet The transformet approach

In [1]:
from tabpfn import TabPFNClassifier, TabPFNRegressor


In [2]:
tab_reg = TabPFNRegressor(device="auto",random_state=42,n_jobs=12)
multioutput_tab_reg = MultiOutputRegressor(estimator=tab_reg)

NameError: name 'MultiOutputRegressor' is not defined

In [ ]:
multioutput_tab_reg.fit(train_X, train_y)

In [ ]:
import joblib

# Save to file
joblib.dump(multioutput_tab_reg, 'multioutput_tabpfn_model.pkl')


In [ ]:
# Load from file
loaded_model = joblib.load('multioutput_tabpfn_model.pkl')

# Now you can use loaded_model.predict(...)
preds = loaded_model.predict(val_X)


In [ ]:
tab_reg_pred = multioutput_tab_reg.predict(val_X)
print("MSE: ", mean_squared_error(val_y, tab_reg_pred))
print("MAE: ", mean_absolute_error(val_y, tab_reg_pred))
print("R2 : ", r2_score(val_y, tab_reg_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, tab_reg_pred))


In [ ]:
tab_reg_pred = multioutput_tab_reg.predict(test_dataset)
tab_reg_pred = pd.DataFrame(tab_reg_pred,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
tab_reg_pred.to_csv('_output_xgb.csv')
tab_reg_pred

In [34]:
# {'estimator__colsample_bytree': 0.982627653632069, 'estimator__learning_rate': 0.1822102743060054, 'estimator__max_bin': 43, 'estimator__max_depth': 17, 'estimator__max_leaves': 15, 'estimator__n_estimators': 94, 'estimator__reg_alpha': 0.10781820337059697, 'estimator__reg_lambda': 0.31170074035318485}
# reg_alpha

In [210]:
from scipy.stats import randint, uniform
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from xgboost import XGBRegressor

# 1) define your base XGB
xgbr = XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    device='cuda',
)

# 2) wrap it
multi_output_model_xgbr = MultiOutputRegressor(estimator=xgbr)

# 3) define your scorer
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# 4) tune using estimator__… prefixes
param_distributions = {
    'estimator__n_estimators': randint(50, 500),
    'estimator__max_depth': randint(5, 50),
    'estimator__learning_rate': uniform(0.001, 0.2),
    'estimator__gamma': uniform(0, 1),
    'estimator__min_child_weight': randint(1, 20),
    'estimator__subsample': uniform(0.5, 1.0),
    'estimator__colsample_bytree': uniform(0.3, 1.0),
    'estimator__reg_alpha': uniform(0, 1),
    'estimator__reg_lambda': uniform(0.1, 5),
}

searcher_xgbr = RandomizedSearchCV(
    multi_output_model_xgbr,
    param_distributions,
    n_iter=40,
    scoring=scorer,
    n_jobs=12,
    cv=3,
    refit=True,
    random_state=42,
)


searcher_xgbr.fit(train_X, train_y)

print("Best hyper-parameters:", searcher.best_params_)
best_model_xgbr = searcher.best_estimator_
# best_model_xgbr = multi_output_model_xgbr
# best_model_xgbr.fit(train_X,train_y)
# 6) evaluate
xgb_pred = best_model_xgbr.predict(val_X)
print("MSE: ", mean_squared_error(val_y, xgb_pred))
print("MAE: ", mean_absolute_error(val_y, xgb_pred))
print("R2 : ", r2_score(val_y, xgb_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, xgb_pred))


MSE:  0.04818769544363022
MAE:  0.15230965614318848
R2 :  0.9503563046455383
MAPE: 0.523072361946106


In [212]:
# print(search.best_params_)
print("MSE: ", mean_squared_error(val_y, xgb_pred))
print("MAE: ", mean_absolute_error(val_y, xgb_pred))
print("R2 : ", r2_score(val_y, xgb_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, xgb_pred))

# {'estimator__colsample_bytree': np.float64(0.5619286433204139), 'estimator__learning_rate': np.float64(0.14791684291885698), 'estimator__max_bin': 136, 'estimator__max_depth': 286, 'estimator__max_leaves': 30, 'estimator__n_estimators': 649, 'estimator__reg_alpha': np.float64(0.2679130529974323), 'estimator__reg_lambda': np.float64(0.90330309864098)}
# MSE:  0.02499525621533394
# MAE:  0.12181054055690765
# R2 :  0.6237972974777222
# MAPE: 0.8065568208694458

MSE:  0.04818769544363022
MAE:  0.15230965614318848
R2 :  0.9503563046455383
MAPE: 0.523072361946106


In [213]:
test_predxgb = best_model_xgbr.predict(test_dataset)
test_predxgb = pd.DataFrame(test_predxgb,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_predxgb.to_csv('_output_xgb.csv')

In [215]:
test_predxgb

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,-0.015244,0.262561,0.517343,0.628340,0.358638,1.027012,0.602140,0.454181,-0.346393,0.482674
2,-0.503274,-0.108632,-1.111737,-0.048354,-0.732576,-0.016033,-1.186800,-0.764654,-0.342960,0.103491
3,1.320022,0.630711,1.008191,1.278908,2.297107,1.785349,1.131096,1.570785,0.242328,2.159818
4,-0.255523,0.432711,0.686053,-0.540557,1.892198,-0.662413,0.494164,1.975723,0.886944,-0.883624
5,-0.012929,-1.319909,1.193228,0.414189,2.389891,0.102231,1.123851,-0.166232,-0.442922,0.767073
...,...,...,...,...,...,...,...,...,...,...
497,-1.986611,-1.026526,-0.986787,-2.126143,-0.632607,-1.955454,-0.883258,-1.223638,-1.443015,-1.388638
498,1.833557,1.417458,0.371863,1.165604,-0.000912,0.661321,0.177298,1.383573,0.389407,0.437213
499,-0.126902,0.319782,1.427497,-0.630168,-0.910105,-0.216748,1.549924,0.370626,-0.324376,0.989642
500,-0.995033,-1.965410,-1.982336,-1.652951,-0.025811,-1.770842,-1.924546,-1.745815,-2.238289,-0.110816


In [216]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

# 1) build a single-output pipeline
pipeline_lr = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    VarianceThreshold(threshold=0.0),
    SelectKBest(score_func=f_regression, k=200),
    StandardScaler(),
    LinearRegression(fit_intercept=True, copy_X=True, n_jobs=12)
)
# 2) wrap in MultiOutputRegressor
multi_output_lr = MultiOutputRegressor(pipeline_lr)

# 3) scorer
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# 4) param grid must reference the pipeline *inside* the wrapper:
param_distributions = {
    'estimator__simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'estimator__selectkbest__k': [50, 100, 150, 200, 'all'],
    'estimator__linearregression__fit_intercept': [True, False],
    'estimator__linearregression__copy_X': [True, False],
}


# 5) randomized search
searcher_lr = RandomizedSearchCV(
    estimator=multi_output_lr,
    param_distributions=param_distributions,
    n_iter=50,
    scoring=scorer,
    cv=5,
    n_jobs=12,
    verbose=2,
    refit=True,
    random_state=42
)

# 6) fit on train
# searcher_lr.fit(train_X, train_y)

# 7) results
# print("Best parameters:", searcher_lr.best_params_)
# best_lr = searcher_lr.best_estimator_
best_lr = multi_output_lr.fit(train_X,train_y)
# 8) evaluate on hold‑out
lr_pred = best_lr.predict(val_X)
print("MSE :", mean_squared_error(val_y, lr_pred))
print("MAE :", mean_absolute_error(val_y, lr_pred))
print("R2  :", r2_score(val_y, lr_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, lr_pred))


MSE : 0.026251269121302174
MAE : 0.0871646572008396
R2  : 0.9779925671297043
MAPE: 0.2317236142965169


In [218]:
# print("Best parameters:", searcher_lr.best_params_)
print("MSE :", mean_squared_error(val_y, lr_pred))
print("MAE :", mean_absolute_error(val_y, lr_pred))
print("R2  :", r2_score(val_y, lr_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, lr_pred))
# Best parameters: {'estimator__simpleimputer__strategy': 'mean', 'estimator__linearregression__fit_intercept': False, 'estimator__linearregression__copy_X': True}
# MSE : 0.06548908205715132
# MAE : 0.11981774831745469
# R2  : 0.6284936932298283
# MAPE: 0.2448504736913187

MSE : 0.026251269121302174
MAE : 0.0871646572008396
R2  : 0.9779925671297043
MAPE: 0.2317236142965169


In [219]:
test_predlr = best_lr.predict(test_dataset)
test_predlr = pd.DataFrame(test_predlr,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_predlr.to_csv('output_lr_1.csv')

In [220]:
test_predlr

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,0.224910,0.082504,0.629089,0.627720,-0.125177,0.713306,0.551139,0.370261,-0.251640,0.330335
2,-0.763007,-0.455490,-1.221844,0.062598,-0.993053,-0.103817,-1.215618,-0.977563,-0.714895,0.015806
3,1.661930,1.171693,1.295254,1.126826,1.842358,1.861225,1.316747,2.029257,0.540062,2.236125
4,-0.348289,0.318713,0.722183,-0.653837,1.812069,-0.438893,0.680427,1.605846,0.434784,-0.958217
5,0.044034,-1.282945,1.073479,0.431034,2.672367,0.237323,1.091580,-0.018463,-0.252148,1.029850
...,...,...,...,...,...,...,...,...,...,...
497,-2.087534,-1.319204,-0.935027,-2.388929,-0.899897,-2.460532,-0.867359,-2.067978,-1.276185,-1.347306
498,1.927030,2.237208,0.319244,1.245595,0.689466,0.660361,0.292052,1.023269,0.330319,0.441129
499,-0.100104,0.672716,1.475961,-1.297491,-0.662134,0.177179,1.584131,0.694500,0.074675,1.282606
500,-1.104785,-1.888859,-2.015209,-1.651944,0.105843,-1.818342,-2.018753,-1.967729,-2.089418,-0.219550


In [221]:
pipeline_rid = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    VarianceThreshold(),        # drop zero‐ or near‐zero‐variance features
    PCA(),        
    Ridge(
        solver='auto',
        random_state=42,
        alpha=np.float64(0.08066305219717405),
    )
)
param_distributions_rid = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'variancethreshold__threshold': [0.0, 1e-3, 1e-2],              # test dropping features with var < threshold
    'pca__n_components': [0.90, 0.95, 0.99, None],                  # keep 90%,95%,99% variance (None ⇒ skip PCA)
    'pca__svd_solver': ['auto', 'full'],                            # solver for PCA
    'ridge__alpha': uniform(0.01, 20),                              # widen alpha range
    'ridge__solver': ['auto', 'sag', 'saga', 'lsqr'],               # focus on solvers that scale well
}

searcher_rid = RandomizedSearchCV(
    estimator=pipeline_rid,
    param_distributions=param_distributions_rid,
    n_iter=50,               
    scoring='neg_mean_absolute_percentage_error',  
    cv=3,
    n_jobs=12,
    verbose=1,
    # random_state=42,
    refit=True
)


# search_rid = searcher_rid.fit(train_X, train_y)
# print("Best params:", search_rid.best_params_)
# best_ridge_pipeline = search_rid.best_estimator_
best_ridge_pipeline=pipeline_rid.fit(train_X,train_y)
rid_pred = best_ridge_pipeline.predict(val_X)

print("MSE:  ", mean_squared_error(val_y, rid_pred))
print("MAE:  ", mean_absolute_error(val_y, rid_pred))
print("R2:   ", r2_score(val_y, rid_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rid_pred))


MSE:   0.030761301666411493
MAE:   0.08956762948810089
R2:    0.9742496065120052
MAPE:  0.21135214227484894


In [222]:
print("Best params:", search_rid.best_params_)
print("MSE:  ", mean_squared_error(val_y, rid_pred))
print("MAE:  ", mean_absolute_error(val_y, rid_pred))
print("R2:   ", r2_score(val_y, rid_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rid_pred))

Best params: {'pca__n_components': None, 'pca__svd_solver': 'auto', 'ridge__alpha': np.float64(0.8695363731325279), 'ridge__solver': 'lsqr', 'simpleimputer__strategy': 'most_frequent', 'variancethreshold__threshold': 0.0}
MSE:   0.030761301666411493
MAE:   0.08956762948810089
R2:    0.9742496065120052
MAPE:  0.21135214227484894


In [223]:
test_pred_rid = best_ridge_pipeline.predict(test_dataset)
test_pred_rid = pd.DataFrame(test_pred_rid,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_rid.to_csv('_output_rid.csv')

In [224]:
test_pred_rid

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,0.229077,0.170115,0.519217,0.607620,-0.316232,0.717484,0.398472,0.298828,-0.132855,0.339114
2,-0.772294,-0.346029,-1.234272,0.040157,-0.887131,-0.125643,-1.306788,-1.046222,-0.806046,-0.009038
3,1.669889,0.867057,1.374719,1.108127,1.894452,1.883224,1.385166,2.105888,0.572858,2.231591
4,-0.353480,0.455931,0.931278,-0.696290,1.569793,-0.452942,0.884242,1.612383,0.343139,-0.975849
5,0.051669,-1.233672,1.095173,0.438265,2.588739,0.252874,1.090350,-0.077851,-0.242066,1.063490
...,...,...,...,...,...,...,...,...,...,...
497,-2.094510,-1.315897,-1.024256,-2.409201,-1.159432,-2.477225,-0.950165,-2.158833,-1.152386,-1.311004
498,1.933581,2.129468,0.333606,1.268938,0.966377,0.678442,0.330093,1.085057,0.337697,0.472938
499,-0.098094,0.717731,1.630418,-1.308743,-0.709099,0.178836,1.697456,0.751087,0.135240,1.295696
500,-1.107943,-1.822103,-2.090734,-1.643663,0.083505,-1.822497,-2.114817,-2.015279,-2.230847,-0.203368


In [167]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, regularizers, callbacks
# from tensorflow.keras.optimizers import AdamW

# def build_strong_tabular_model(
#     n_features=579, 
#     n_targets=10,
#     embed_dim=32,
#     transformer_heads=4,
#     transformer_ff=128,
#     depth_residual=3,
#     l2=1e-4,
#     dropout=0.3
# ):
#     inp = layers.Input(shape=(n_features,), name="features")
#     # 1) Per‑feature embedding
#     x = layers.Reshape((n_features, 1))(inp)
#     x = layers.Conv1D(embed_dim, 1, 
#                       kernel_regularizer=regularizers.l2(l2),
#                       name="feature_embedding")(x)

#     # 2) Single Transformer encoder block
#     attn = layers.MultiHeadAttention(transformer_heads, embed_dim,
#                                      kernel_regularizer=regularizers.l2(l2),
#                                      name="mha")(x, x)
#     attn = layers.Dropout(dropout)(attn)
#     attn = layers.Add()([x, attn])
#     attn = layers.LayerNormalization()(attn)

#     ff = layers.Dense(transformer_ff, activation="relu",
#                       kernel_regularizer=regularizers.l2(l2))(attn)
#     ff = layers.Dropout(dropout)(ff)
#     ff = layers.Dense(embed_dim,
#                       kernel_regularizer=regularizers.l2(l2))(ff)
#     x = layers.Add()([attn, ff])
#     x = layers.LayerNormalization(name="transformer_out")(x)

#     # 3) Flatten
#     x = layers.Flatten()(x)  # → (batch, 579*32)

#     # 4) Residual Dense blocks with projection of shortcut → 256 dims
#     for i in range(depth_residual):
#         # project the shortcut to 256 dims
#         shortcut = layers.Dense(
#             256, activation=None,
#             kernel_regularizer=regularizers.l2(l2),
#             name=f"shortcut_proj_{i}"
#         )(x)
        
#         x = layers.Dense(
#             256, activation="relu",
#             kernel_regularizer=regularizers.l2(l2),
#             name=f"res_dense_{i}_1"
#         )(x)
#         x = layers.Dropout(dropout)(x)
#         x = layers.Dense(
#             256, activation=None,
#             kernel_regularizer=regularizers.l2(l2),
#             name=f"res_dense_{i}_2"
#         )(x)
        
#         x = layers.Add(name=f"res_add_{i}")([shortcut, x])
#         x = layers.LayerNormalization(name=f"res_norm_{i}")(x)

#     # 5) Final head
#     x = layers.Dense(
#         128, activation="relu",
#         kernel_regularizer=regularizers.l2(l2),
#         name="head_dense"
#     )(x)
#     x = layers.Dropout(dropout)(x)
#     outputs = layers.Dense(
#         n_targets, activation="linear",
#         name="outputs"
#     )(x)

#     model = models.Model(inputs=inp, outputs=outputs, name="StrongTabNet")
#     model.compile(
#         optimizer=AdamW(learning_rate=3e-4, weight_decay=1e-5),
#         loss="mean_absolute_percentage_error",
#         metrics=[
#             tf.keras.metrics.MeanAbsoluteError(name="mae"),
#             tf.keras.metrics.RootMeanSquaredError(name="rmse"),
#             tf.keras.metrics.MeanAbsolutePercentageError(name='mean_absolute_percentage_error')
#         ]
#     )
#     return model

# # --- Usage ---
# model = build_strong_tabular_model(
#     n_features=579,
#     n_targets=10,
#     embed_dim=16,
#     transformer_heads=1,
#     transformer_ff=128,
#     depth_residual=1,
#     l2=1e-4,
#     dropout=0.4
# )
# model.summary()




In [168]:
# history = model.fit(
#     train_X, train_y,
#     validation_data=(val_X, val_y),
#     epochs=200,
#     batch_size=32,
#     callbacks=[
#         callbacks.EarlyStopping(patience=10, restore_best_weights=True),
#         callbacks.ReduceLROnPlateau(factor=0.5, patience=10, min_lr=1e-6)
#     ]
# )

In [169]:
# mlp_pred = model.predict(val_X)

In [170]:
# print("MSE:  ", mean_squared_error(val_y,  mlp_pred))
# print("MAE:  ", mean_absolute_error(val_y,  mlp_pred))
# print("R2:   ", r2_score(val_y,mlp_pred))
# print("MAPE: ", mean_absolute_percentage_error(val_y, mlp_pred))

In [171]:
# test_pred_mlp = model.predict(test_dataset)
# test_pred_mlp = np.expm1(test_pred_mlp)
# test_pred_mlp = pd.DataFrame(test_pred_mlp,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
#        'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
#        'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
# test_pred_mlp.to_csv('_output_mlp.csv')

In [230]:
pipeline_rf = make_pipeline(
    SimpleImputer(strategy='mean'),
    RandomForestRegressor(    # baki hai
        # n_estimators=378,
        # criterion='squared_error',
        # max_depth=232,
        # min_samples_split=218,
        # min_samples_leaf=498,
        # max_features='log2',
        # bootstrap=True,
        # n_jobs=12,
        # random_state=42,
        # verbose=0,
        # warm_start=False,
        # max_leaf_nodes= 155,
    )
)

# Best RF params: {'randomforestregressor__criterion': 'squared_error', 'randomforestregressor__max_depth': 232, 'randomforestregressor__max_features': 'log2', 'randomforestregressor__max_leaf_nodes': 155, 'randomforestregressor__min_samples_leaf': 498, 'randomforestregressor__min_samples_split': 218, 'randomforestregressor__n_estimators': 378, 'simpleimputer__strategy': 'mean'}
# MSE:   1.1939122283517236
# MAE:   0.8203986469907789
# R²:    -10.916110902599737
# MAPE:  1.04675604732512

param_distributions_rf = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'randomforestregressor__criterion': ["squared_error", "absolute_error", "friedman_mse", "poisson"],
    'randomforestregressor__n_estimators': randint(3, 1000),
    'randomforestregressor__min_samples_split': randint(2, 500),
    'randomforestregressor__min_samples_leaf': randint(1, 500),
    'randomforestregressor__max_features': ["sqrt", "log2", None],
    'randomforestregressor__max_leaf_nodes': randint(3, 1000),
    'randomforestregressor__max_depth': randint(3, 800),
}

searcher_rf = RandomizedSearchCV(
    estimator=pipeline_rf,
    param_distributions=param_distributions_rf,
    n_iter=10,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=3,
    n_jobs=-1,
    verbose=2, 
    # random_state=42,
    refit=True
)

# search_rf = searcher_rf.fit(train_X, train_y)

# print("Best RF params:", search_rf.best_params_)

# best_rf = search_rf.best_estimator_
best_rf = pipeline_rf.fit(train_X,train_y)
rf_pred = best_rf.predict(val_X)

# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, rf_pred))
print("MAE:  ", mean_absolute_error(val_y, rf_pred))
print("R²:   ", r2_score(val_y, rf_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rf_pred))


MSE:   0.28220199051659706
MAE:   0.3924586636900017
R²:    0.7168658016548453
MAPE:  1.8766797900657033


In [231]:
# print("Best RF params:", search_rf.best_params_)
print("MSE:  ", mean_squared_error(val_y, rf_pred))
print("MAE:  ", mean_absolute_error(val_y, rf_pred))
print("R²:   ", r2_score(val_y, rf_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rf_pred))

MSE:   0.28220199051659706
MAE:   0.3924586636900017
R²:    0.7168658016548453
MAPE:  1.8766797900657033


In [232]:
test_pred_rf = best_rf.predict(test_dataset)
test_pred_rf = pd.DataFrame(test_pred_rf,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_rf.to_csv('_output_rf.csv')

In [233]:
test_pred_rf

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,-0.307956,0.028171,0.518988,0.031838,-0.364729,0.204608,0.507216,0.092328,-0.077430,0.044509
2,-0.385525,-0.326183,-1.320598,-0.535185,-0.512064,-0.270405,-1.305770,-0.918932,-0.013287,-0.223707
3,1.515444,1.194495,1.161984,1.400606,1.524938,1.598896,1.166225,1.467000,0.897544,1.862588
4,0.618601,0.453147,0.533429,0.359348,1.612693,0.164932,0.517472,0.656033,0.168014,-0.417537
5,-0.102677,-0.051193,1.087263,0.102316,2.229729,0.280512,1.088986,0.448957,-0.287292,0.705247
...,...,...,...,...,...,...,...,...,...,...
497,-1.455314,-1.331177,-1.311355,-1.339578,-0.292081,-1.352342,-1.296734,-1.528966,-1.200365,-1.259780
498,0.978024,0.833813,0.113734,1.034283,0.064944,0.600163,0.110828,0.569078,0.702694,-0.090700
499,0.254644,0.339155,0.675556,0.365914,-0.430724,-0.041492,0.653085,0.223081,-0.194637,0.982603
500,-1.543401,-1.725081,-1.822962,-1.585764,-0.466999,-1.831542,-1.798762,-1.691150,-1.789837,-0.539956


In [241]:
pipeline_etc = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    ExtraTreesRegressor(
        # n_estimators=100,
        # criterion='squared_error',
        # max_depth=515,
        # min_samples_split=217,
        # min_samples_leaf=226,
        # max_features='log2',
        # bootstrap=True,
        # random_state=42,
        # ccp_alpha = np.float64(0.4911411839915161),
        # n_jobs=12,
        # warm_start=False
    )
)
# Best ExtraTrees params: {'extratreesregressor__bootstrap': True, 'extratreesregressor__ccp_alpha': np.float64(0.4911411839915161), 'extratreesregressor__max_depth': 515, 'extratreesregressor__max_features': 'log2', 'extratreesregressor__min_samples_leaf': 226, 'extratreesregressor__min_samples_split': 217, 'extratreesregressor__n_estimators': 100, 'simpleimputer__strategy': 'most_frequent'}
# MSE:   1.3215288190857684
# MAE:   0.869117328594581
# R²:    -13.578031179018907
# MAPE:  0.9793419452393474

param_distributions_etc = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'extratreesregressor__n_estimators': randint(5, 800),
    'extratreesregressor__max_depth': [None] + list(range(5, 800, 5)),
    'extratreesregressor__min_samples_split': randint(2, 500),
    'extratreesregressor__min_samples_leaf': randint(1, 500),
    'extratreesregressor__max_features': ['sqrt', 'log2', None] + list(uniform(0.1, 0.9).rvs(5)),
    'extratreesregressor__bootstrap': [False, True],
    'extratreesregressor__ccp_alpha': uniform(0.0, 0.9),  # cost-complexity pruning
}

searcher_etc = RandomizedSearchCV(
    estimator=pipeline_etc,
    param_distributions=param_distributions_etc,
    n_iter=100,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=5,
    n_jobs=12,
    verbose=0,
    random_state=42,
    refit=True
)

# search_etc = searcher_etc.fit(train_X, train_y)

# print("Best ExtraTrees params:", search_etc.best_params_)

# best_etc = search_etc.best_estimator_
best_etc = pipeline_etc.fit(train_X,train_y)
etc_pred = best_etc.predict(val_X)

# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, etc_pred))
print("MAE:  ", mean_absolute_error(val_y, etc_pred))
print("R²:   ", r2_score(val_y, etc_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, etc_pred))


MSE:   0.2503833363230411
MAE:   0.37851161539666406
R²:    0.7509420320949337
MAPE:  1.9154950202923442


In [242]:
# print("Best ExtraTrees params:", search_etc.best_params_)
print("MSE:  ", mean_squared_error(val_y, etc_pred))
print("MAE:  ", mean_absolute_error(val_y, etc_pred))
print("R²:   ", r2_score(val_y, etc_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, etc_pred))


MSE:   0.2503833363230411
MAE:   0.37851161539666406
R²:    0.7509420320949337
MAPE:  1.9154950202923442


In [243]:
test_pred_etc = best_etc.predict(test_dataset)
test_pred_etc = pd.DataFrame(test_pred_etc,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_etc.to_csv('_output_etc.csv')

In [244]:
test_pred_etc

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,-0.073642,0.121881,0.483780,0.057991,-0.454720,0.129093,0.468141,0.201843,-0.158845,-0.013985
2,-0.411457,-0.340469,-1.362809,-0.458975,-0.432283,-0.253554,-1.347183,-0.951138,0.046413,-0.143959
3,1.496904,1.005987,1.097957,1.293527,1.556718,1.569147,1.097127,1.471172,0.818330,1.910572
4,0.616205,0.448388,0.499421,0.158649,1.611792,0.229114,0.480240,0.671863,0.303039,-0.522912
5,-0.138135,0.052650,1.169970,-0.146372,2.421059,0.237528,1.185157,0.726699,-0.319427,0.771009
...,...,...,...,...,...,...,...,...,...,...
497,-1.580711,-1.393766,-1.249730,-1.331598,-0.283079,-1.100229,-1.236255,-1.663971,-1.244208,-1.443307
498,1.091895,0.968594,0.465112,1.119162,0.056313,0.859883,0.455883,0.818247,0.710104,0.143049
499,0.421667,0.294933,0.871226,0.448943,-0.571537,0.206579,0.867861,0.501508,0.009039,0.634405
500,-1.441017,-1.688570,-1.920537,-1.573250,-0.486900,-1.745817,-1.894509,-1.678698,-2.051507,-0.446456


In [248]:
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.ensemble import StackingRegressor

# # 1) single‐output stacking (each estimator here must be single‐output)
# base_estimators = [
#     ('xgb', xgbr),
#     ('lr',  pipeline_lr),
#     # ('rf',  pipeline_rf),
#     ('etc', pipeline_etc),
# ]

# stack_single = StackingRegressor(
#     estimators=base_estimators,
#     final_estimator=pipeline_rid,  # pipeline_rid is SimpleImputer+Ridge
#     cv=2,
#     n_jobs=12
# )

# # 2) wrap the entire stack
# stack_multi = MultiOutputRegressor(stack_single)

# # 3) fit & predict
# stack_multi.fit(train_X, train_y)       # train_y shape = (n_samples, n_targets)
# y_pred_stack = stack_multi.predict(val_X)


KeyboardInterrupt: 

In [ ]:
print("MSE :", mean_squared_error(val_y, y_pred_stack))
print("MAE :", mean_absolute_error(val_y, y_pred_stack))
print("R2  :", r2_score(val_y, y_pred_stack))
print("MAPE:", mean_absolute_percentage_error(val_y, y_pred_stack))


In [ ]:
test_pred_stack = stack_multi.predict(test_dataset)
test_pred_stack = pd.DataFrame(test_pred_stack,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_stack.to_csv('output_check_1_poly.csv')

In [249]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Define the pipeline
from sklearn.preprocessing import FunctionTransformer
pipeline_svr = make_pipeline(
    SimpleImputer(strategy= 'median'),
    SVR()
)


multioutput_svr = MultiOutputRegressor(pipeline_svr)

# Correct parameter grid with proper prefix
param_distributions_svr = {
    'estimator__simpleimputer__strategy': ['mean','median'],
    'estimator__standardscaler': [StandardScaler()],
    'estimator__svr__kernel': ['rbf','linear','poly'],
    'estimator__svr__C': [0.1, 1, 10, 100],
    'estimator__svr__gamma': ['scale', 0.01, 0.1, 1],
    'estimator__svr__epsilon': [0.01, 0.1, 0.5, 1],
    'estimator__svr__degree': [2,3,4]  # only used if kernel='poly'
}

searcher_svr = RandomizedSearchCV(
    estimator=multioutput_svr,
    param_distributions=param_distributions_svr,
    n_iter=50,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=3,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    refit=True
)

# search_svr = searcher_svr.fit(train_X, train_y)

# print("Best SVR params:", search_svr.best_params_)

# best_svr = search_svr.best_estimator_
best_svr = multioutput_svr.fit(train_X,train_y)
svr_pred = best_svr.predict(val_X)

# Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, svr_pred))
print("MAE:  ", mean_absolute_error(val_y, svr_pred))
print("R²:   ", r2_score(val_y, svr_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, svr_pred))


MSE:   1.034641394755515
MAE:   0.8417113509939739
R²:    -0.018618887231208737
MAPE:  1.0759424501684012


In [250]:
test_pred_svr = best_svr.predict(test_dataset)
test_pred_svr = pd.DataFrame(test_pred_svr,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_svr.to_csv('output_check_1_poly.csv')

In [251]:
test_pred_svr

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,-0.000297,0.021240,0.152907,-0.014182,-0.251645,-0.022483,0.141740,-0.005312,-0.003052,-0.028097
2,0.006043,0.098951,0.181646,-0.018109,-0.208133,-0.002639,0.170210,0.023989,0.013854,0.010096
3,-0.021517,-0.002950,0.055852,-0.085544,-0.276607,-0.040575,0.045102,-0.069568,-0.034165,-0.021929
4,-0.002700,0.057559,0.131767,-0.055193,-0.248111,-0.015907,0.120950,-0.017166,0.014602,0.015068
5,0.010368,0.085605,0.159937,-0.016086,-0.226344,-0.025267,0.148380,0.046531,0.004692,0.016019
...,...,...,...,...,...,...,...,...,...,...
497,-0.003271,0.025402,0.198794,0.009290,-0.229242,-0.050170,0.187107,-0.019450,-0.025119,-0.055584
498,-0.005111,0.040229,0.086612,-0.058410,-0.242365,-0.005573,0.075758,-0.011462,-0.007679,0.011447
499,-0.009293,0.048693,0.169725,-0.053390,-0.219697,-0.025565,0.158317,0.005900,0.004415,0.033157
500,0.099476,0.176031,0.076193,0.102609,-0.226454,0.206841,0.065816,-0.002152,0.125242,-0.018734


In [ ]:
# !pip install catboost

In [253]:
from catboost import CatBoostRegressor


pipeline_cat = make_pipeline(
    SimpleImputer(),
        CatBoostRegressor(
            # iterations=1000,
            # learning_rate=0.001,
            # depth=6,
            # silent=True,
            # random_state=42,
            # thread_count=-1,
            # task_type='GPU',
            # devices='0',
            # bootstrap_type='Bayesian'  # Default for GPU, robust
        )
)
multioutput_cat = MultiOutputRegressor(pipeline_cat)

param_distributions_cat = {
    'simpleimputer__strategy': ['mean', 'median'],
    'multioutputregressor__estimator__iterations': randint(100, 2000),
    'multioutputregressor__estimator__learning_rate': uniform(0.01, 0.3),
    'multioutputregressor__estimator__depth': randint(4, 10),
    'multioutputregressor__estimator__l2_leaf_reg': uniform(1, 10),
    'multioutputregressor__estimator__rsm': uniform(0.5, 0.5),
    'multioutputregressor__estimator__border_count': randint(32, 255),
    # 'multioutputregressor__estimator__subsample': uniform(0.5, 0.5),  # REMOVE THIS LINE
}

searcher_cat = RandomizedSearchCV(
    estimator=pipeline_cat,
    param_distributions=param_distributions_cat,
    n_iter=10,
    scoring='neg_mean_absolute_percentage_error',
    cv=3,
    n_jobs=12,
    verbose=2,
    random_state=42,
    refit=True
)

# search_cat = searcher_cat.fit(train_X, train_y)

# print("Best CatBoost params:", search_cat.best_params_)

# best_cat = search_cat.best_estimator_
best_cat = multioutput_cat
best_cat.fit(train_X,train_y)
cat_pred = best_cat.predict(val_X)

print("MSE:  ", mean_squared_error(val_y, cat_pred))
print("MAE:  ", mean_absolute_error(val_y, cat_pred))
print("R²:   ", r2_score(val_y, cat_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, cat_pred))


Learning rate set to 0.045609
0:	learn: 0.9629194	total: 81.3ms	remaining: 1m 21s
1:	learn: 0.9349318	total: 106ms	remaining: 52.9s
2:	learn: 0.9091923	total: 128ms	remaining: 42.6s
3:	learn: 0.8862881	total: 152ms	remaining: 38s
4:	learn: 0.8612305	total: 176ms	remaining: 35.1s
5:	learn: 0.8353034	total: 200ms	remaining: 33.2s
6:	learn: 0.8151677	total: 225ms	remaining: 31.9s
7:	learn: 0.7931758	total: 251ms	remaining: 31.1s
8:	learn: 0.7736636	total: 275ms	remaining: 30.2s
9:	learn: 0.7514549	total: 299ms	remaining: 29.6s
10:	learn: 0.7360521	total: 323ms	remaining: 29.1s
11:	learn: 0.7168749	total: 347ms	remaining: 28.6s
12:	learn: 0.6985545	total: 369ms	remaining: 28s
13:	learn: 0.6838627	total: 391ms	remaining: 27.5s
14:	learn: 0.6687907	total: 414ms	remaining: 27.2s
15:	learn: 0.6519564	total: 438ms	remaining: 26.9s
16:	learn: 0.6369920	total: 463ms	remaining: 26.8s
17:	learn: 0.6238313	total: 487ms	remaining: 26.6s
18:	learn: 0.6084534	total: 511ms	remaining: 26.4s
19:	learn: 0.

In [254]:
# print("Best CatBoost params:", search_cat.best_params_)
print("MSE:  ", mean_squared_error(val_y, cat_pred))
print("MAE:  ", mean_absolute_error(val_y, cat_pred))
print("R²:   ", r2_score(val_y, cat_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, cat_pred))

MSE:   0.023727950451015452
MAE:   0.09508058118988516
R²:    0.979111510421656
MAPE:  0.31273135535941404


In [255]:
test_pred_cat = best_svr.predict(test_dataset)
test_pred_cat = pd.DataFrame(test_pred_cat,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_cat.to_csv('output_check_cat.csv')

In [256]:
test_pred_cat

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,-0.000297,0.021240,0.152907,-0.014182,-0.251645,-0.022483,0.141740,-0.005312,-0.003052,-0.028097
2,0.006043,0.098951,0.181646,-0.018109,-0.208133,-0.002639,0.170210,0.023989,0.013854,0.010096
3,-0.021517,-0.002950,0.055852,-0.085544,-0.276607,-0.040575,0.045102,-0.069568,-0.034165,-0.021929
4,-0.002700,0.057559,0.131767,-0.055193,-0.248111,-0.015907,0.120950,-0.017166,0.014602,0.015068
5,0.010368,0.085605,0.159937,-0.016086,-0.226344,-0.025267,0.148380,0.046531,0.004692,0.016019
...,...,...,...,...,...,...,...,...,...,...
497,-0.003271,0.025402,0.198794,0.009290,-0.229242,-0.050170,0.187107,-0.019450,-0.025119,-0.055584
498,-0.005111,0.040229,0.086612,-0.058410,-0.242365,-0.005573,0.075758,-0.011462,-0.007679,0.011447
499,-0.009293,0.048693,0.169725,-0.053390,-0.219697,-0.025565,0.158317,0.005900,0.004415,0.033157
500,0.099476,0.176031,0.076193,0.102609,-0.226454,0.206841,0.065816,-0.002152,0.125242,-0.018734


In [261]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from scipy.stats import randint, uniform

# Build pipeline
pipeline_gbm = make_pipeline(
    SimpleImputer(),  # fill missing values
    GradientBoostingRegressor(
        # loss='squared_error',
        # learning_rate=0.001,
        # n_estimators=300,
        # subsample=1.0,
        # min_samples_split=10,
        # min_samples_leaf=20,
        # min_weight_fraction_leaf=0.0,
        # max_depth=10,
        # min_impurity_decrease=0.0,
        # init=None,
        # random_state=None,
        # max_features=None,
        # alpha=0.99,
        verbose=2,
        # max_leaf_nodes=500,
        # warm_start=True,
        # validation_fraction=0.1,
        n_iter_no_change=3,
        # tol=0.0001,
        # ccp_alpha=0.0,
    )
)

# Wrap in multioutput
multioutput_gbm = MultiOutputRegressor(pipeline_gbm)

# Parameter grid
param_distributions_gbm = {
    'estimator__simpleimputer__strategy': ['mean', 'median'],
    'estimator__gradientboostingregressor__n_estimators': randint(50, 500),
    'estimator__gradientboostingregressor__learning_rate': uniform(0.01, 0.3),
    'estimator__gradientboostingregressor__max_depth': randint(3, 10),
    'estimator__gradientboostingregressor__subsample': uniform(0.5, 0.5),
    'estimator__gradientboostingregressor__max_features': [1.0, 'sqrt', 'log2', None],
    'estimator__gradientboostingregressor__min_samples_split': randint(2, 20),
    'estimator__gradientboostingregressor__min_samples_leaf': randint(1, 20),
}


# Search
searcher_gbm = RandomizedSearchCV(
    estimator=multioutput_gbm,
    param_distributions=param_distributions_gbm,
    n_iter=50,
    scoring='neg_mean_absolute_percentage_error',
    cv=3,
    n_jobs=12,
    verbose=0,
    # random_state=42,
    refit=True
)

# search_gbm = searcher_gbm.fit(train_X, train_y)

# # Results
# print("Best GBM params:", search_gbm.best_params_)

# best_gbm = search_gbm.best_estimator_
best_gbm = multioutput_gbm
best_gbm.fit(train_X,train_y)
gbm_pred = best_gbm.predict(val_X)

print("MSE:  ", mean_squared_error(val_y, gbm_pred))
print("MAE:  ", mean_absolute_error(val_y, gbm_pred))
print("R²:   ", r2_score(val_y, gbm_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, gbm_pred))


      Iter       Train Loss   Remaining Time 
         1           0.8682           20.24s
         2           0.7640           20.06s
         3           0.6766           19.82s
         4           0.6015           19.58s
         5           0.5348           19.88s
         6           0.4748           19.54s
         7           0.4272           19.26s
         8           0.3831           19.01s
         9           0.3456           18.77s
        10           0.3121           18.55s
        11           0.2824           18.34s
        12           0.2559           18.15s
        13           0.2336           17.93s
        14           0.2139           17.72s
        15           0.1957           17.52s
        16           0.1802           17.33s
        17           0.1659           17.12s
        18           0.1536           16.90s
        19           0.1416           16.71s
        20           0.1306           16.48s
        21           0.1209           16.25s
        2

KeyboardInterrupt: 

In [259]:
print("MSE:  ", mean_squared_error(val_y, gbm_pred))
print("MAE:  ", mean_absolute_error(val_y, gbm_pred))
print("R²:   ", r2_score(val_y, gbm_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, gbm_pred))
test_pred_gbn = best_svr.predict(test_dataset)
test_pred_gbn = pd.DataFrame(test_pred_gbn,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_gbn.to_csv('output_check_gbm.csv')
test_pred_gbn

MSE:   0.6284058107782016
MAE:   0.6585448799362529
R²:    0.3809565653128752
MAPE:  0.8710108552712115


,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
1,-0.000297,0.021240,0.152907,-0.014182,-0.251645,-0.022483,0.141740,-0.005312,-0.003052,-0.028097
2,0.006043,0.098951,0.181646,-0.018109,-0.208133,-0.002639,0.170210,0.023989,0.013854,0.010096
3,-0.021517,-0.002950,0.055852,-0.085544,-0.276607,-0.040575,0.045102,-0.069568,-0.034165,-0.021929
4,-0.002700,0.057559,0.131767,-0.055193,-0.248111,-0.015907,0.120950,-0.017166,0.014602,0.015068
5,0.010368,0.085605,0.159937,-0.016086,-0.226344,-0.025267,0.148380,0.046531,0.004692,0.016019
...,...,...,...,...,...,...,...,...,...,...
497,-0.003271,0.025402,0.198794,0.009290,-0.229242,-0.050170,0.187107,-0.019450,-0.025119,-0.055584
498,-0.005111,0.040229,0.086612,-0.058410,-0.242365,-0.005573,0.075758,-0.011462,-0.007679,0.011447
499,-0.009293,0.048693,0.169725,-0.053390,-0.219697,-0.025565,0.158317,0.005900,0.004415,0.033157
500,0.099476,0.176031,0.076193,0.102609,-0.226454,0.206841,0.065816,-0.002152,0.125242,-0.018734


In [56]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import StackingRegressor

# 1) single‐output stacking (each estimator here must be single‐output)
base_estimators = [
    ('xgb', xgbr),
    ('lr',  pipeline_lr),
    ('svr', pipeline_svr)
    ('rf',  pipeline_rf),
    ('etc', pipeline_etc),
    ('cat', pipeline_cat),
    ('gbm', pipeline_gbm),
]

stack_single = StackingRegressor(
    estimators=base_estimators,
    final_estimator=pipeline_rid,  # pipeline_rid is SimpleImputer+Ridge
    cv=5,
    n_jobs=12
)

# 2) wrap the entire stack
stack_multi = MultiOutputRegressor(stack_single)

# 3) fit & predict
stack_multi.fit(train_X, train_y)       # train_y shape = (n_samples, n_targets)
y_pred_stack = stack_multi.predict(val_X)

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


      Iter       Train Loss   Remaining Time 
         1           0.9850           15.17m
         2           0.9832           14.46m
         3           0.9814           13.95m
         4           0.9796           13.68m
         5           0.9778           13.48m
         6           0.9760           13.29m
         7           0.9743           13.00m
         8           0.9725           12.81m
         9           0.9707           12.58m
        10           0.9690           12.39m
        11           0.9672           12.24m
        12           0.9655           12.15m
        13           0.9637           12.04m
        14           0.9620           11.93m
        15           0.9602           11.82m
        16           0.9585           11.71m
        17           0.9567           11.60m
        18           0.9550           11.52m
        19           0.9533           11.43m
        20           0.9515           11.33m
        21           0.9498           11.24m
        2

[14:58:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880503067/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.



       182           0.7113            4.39m
       183           0.7100            4.35m
       184           0.7088            4.32m
       185           0.7075            4.28m
       186           0.7063            4.24m
       187           0.7050            4.20m
       188           0.7038            4.16m
       189           0.7025            4.12m
       190           0.7012            4.08m
       191           0.7000            4.04m
       192           0.6987            4.00m
       193           0.6975            3.96m
       194           0.6962            3.93m
       195           0.6950            3.89m
       196           0.6938            3.85m
       197           0.6925            3.81m
       198           0.6913            3.77m
       199           0.6900            3.73m
       200           0.6888            3.69m
       201           0.6876            3.66m
       202           0.6864            3.62m
       203           0.6851            3.58m
       204

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 

      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
         1           0.9666           17.54m
         1           0.9564           17.60m
         1           1.0000           17.80m
         1           0.9993           18.60m
         1           1.0017           19.07m
         2           0.9982           16.65m
         2           0.9547           16.60m
         2           0.9649           16.81m
         2           0.9999           17.46m
         2           0.9975           17.55m
         3           0.9964           16.08m
         3           0.9530           16.14m
         3           0.9632           16.29m
         3           0.9957           17.01m
         3           0.9981           17.11m
         4           0.9512           15.98m
         4           0.9615           16.10m
     

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


       290           0.6000           20.72s
       288           0.5766           25.05s
       288           0.6023           25.08s
       294           0.5950           12.30s
       295           0.5753           10.22s
       291           0.5989           18.65s
       289           0.5756           22.96s
       289           0.6012           22.98s
       295           0.5940           10.25s
       296           0.5743            8.17s
       292           0.5979           16.57s
       290           0.5746           20.87s
       290           0.6002           20.89s
       296           0.5929            8.20s
       297           0.5733            6.13s
       293           0.5968           14.50s
       291           0.5736           18.78s
       291           0.5991           18.80s
       297           0.5919            6.15s
       294           0.5958           12.42s
       298           0.5723            4.09s
       292           0.5726           16.69s
       292

KeyboardInterrupt: 

In [ ]:
print("MSE :", mean_squared_error(val_y, y_pred_stack))
print("MAE :", mean_absolute_error(val_y, y_pred_stack))
print("R2  :", r2_score(val_y, y_pred_stack))
print("MAPE:", mean_absolute_percentage_error(val_y, y_pred_stack))

test_pred_stack = stack_multi.predict(test_dataset)
test_pred_stack = pd.DataFrame(test_pred_stack,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_stack.to_csv('output_check_stack.csv')

In [ ]:
# from sklearn.ensemble import VotingRegressor
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# # Ensure your pipelines are single-output (not wrapped in MultiOutput)
# estimators = [
#     ('xgb', xgbr),
#     ('rf',  pipeline_rf),
#     ('etc', pipeline_etc),
#     ('mlp', pipeline_mlp),
#     ('lr',  pipeline_lr),       # single-output pipeline
#     ('rid', pipeline_rid),      # single-output pipeline
#     ('gbm', pipeline_gbm),      # single-output pipeline
# ]

# base_voting = VotingRegressor(estimators=estimators, n_jobs=-1)
# voting = MultiOutputRegressor(base_voting, n_jobs=-1)

# voting.fit(train_X, train_y)

# voting_pred = voting.predict(val_X)
# print("Voting MSE:  ", mean_squared_error(val_y, voting_pred))
# print("Voting MAE:  ", mean_absolute_error(val_y, voting_pred))
# print("Voting R²:   ", r2_score(val_y, voting_pred))
# print("Voting MAPE: ", mean_absolute_percentage_error(val_y, voting_pred))


In [ ]:
# print("MSE :", mean_squared_error(val_y, voting_pred))
# print("MAE :", mean_absolute_error(val_y, voting_pred))
# print("R2  :", r2_score(val_y, voting_pred))
# print("MAPE:", mean_absolute_percentage_error(val_y, voting_pred))

# test_pred_voting = voting.predict(test_dataset)
# test_pred_voting = pd.DataFrame(test_pred_voting,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
#        'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
#        'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
# test_pred_voting.to_csv('output_check_3.csv')

# Applying deeplearning